In [1]:
from sklearn.ensemble import RandomForestClassifier
from Geo_cal_utils import GeometricCalibrator,ECE_calc
from sklearn.model_selection import train_test_split

In [2]:
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

In [3]:
X, y = make_classification(
    n_samples=1000,  # row number
    n_features=900, # because our calibrator is working on images the number of features need to simulate image, thus the number we choose should hav square root.
    n_informative=6, # The number of informative features
    n_classes = 2, # The number of classes
    random_state = 42 # random seed 
)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [5]:
model = RandomForestClassifier().fit(X_train,y_train)

In [6]:
y_pred_test = model.predict(X_test)
y_test_probs = model.predict_proba(X_test)
accuracy_score(y_test, y_pred_test)

0.8

In [7]:
GeoCalibratorAS = GeometricCalibrator(model,X_train,y_train,method="Approx Seperation", comprasion_mode='Maxpool', comprassion_param=2 )
GeoCalibratorAS.fit(X_val,y_val)
calibrated_prob_AS = GeoCalibratorAS.calibrate(X_test,y_test)

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [8]:
GeoCalibratorS = GeometricCalibrator(model,X_train,y_train,method="Seperation")
GeoCalibratorS.fit(X_val,y_val)
calibrated_prob_S = GeoCalibratorS.calibrate(X_test,y_test)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [12]:
# After Calibration
ECE_calc(calibrated_prob_AS,y_pred_test,y_test)

0.04267563926621227

In [13]:
# After Calibration
ECE_calc(calibrated_prob_S,y_pred_test,y_test)

0.056671466569510824

In [14]:
# before Calibration
GeoCalibratorAS.ECE(y_test_probs,y_pred_test,y_test)

0.2096000000000001